In [ ]:
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from src.models.models import LRIUnet, Unet
from src.models.loss import dice_coe_loss, dice_coe_metric
from src.data.drive import (get_dataset, tf_random_crop, tf_random_rotate)

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [ ]:
ds = get_dataset().cache().map(tf_random_crop).map(lambda x, y, z:
                                                   (x, z)).batch(1)


In [ ]:
x, y_true = next(ds.as_numpy_iterator())

In [ ]:
plt.subplot(121)
plt.imshow(x[0, ...])
plt.subplot(122)
plt.imshow(y_true[0, ...])


In [ ]:
model = LRIUnet(output_channels=1, kind="bispectrum")
optimizer = tf.keras.optimizers.Adam(1e-3)

In [ ]:
with tf.GradientTape() as tape:
  # Forward pass
  y = model(x, training=True)
  loss = dice_coe_loss(y_true, y)


In [ ]:
model.up_stack[0].trans_conv

In [ ]:
model.down_stack[1].get_layer("residual_lri_layer2d_16").conv

In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=True, dpi=96
)


In [ ]:
grads = tape.gradient(loss, model.trainable_variables)

In [ ]:
nans = np.array([np.any(np.isnan(g)) for g in grads])
np.sum(nans)

In [ ]:
nans = np.array([np.any(np.isnan(g)) for g in model.trainable_weights])
np.sum(nans)

In [ ]:
epochs = 1
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch, ))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(ds):

        with tf.GradientTape() as tape:
            y_pred = model(x_batch_train, training=True)
            loss_value = dice_coe_loss(y_batch_train, y_pred)

        grads = tape.gradient(loss_value, model.trainable_weights)

        optimizer.apply_gradients(zip(grads, model.trainable_weights))



In [ ]:
model.trainable_weights